In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import pymysql
import datetime
plt.style.use('fivethirtyeight')
#plt.style.use('default')

from functools import reduce


pd.set_option('display.max_columns',None)
pd.set_option('display.min_rows', None)

In [2]:
df_account = pd.read_csv('df_account.csv') 
df_client = pd.read_csv('df_client.csv')
df_disp = pd.read_csv('df_disp.csv')
df_order = pd.read_csv('df_order.csv')
df_trans = pd.read_csv('df_trans.csv')
df_loan = pd.read_csv('df_loan.csv')
df_card = pd.read_csv('df_card.csv')
df_district = pd.read_csv('df_district.csv')

/Users/gthorani/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_order.head()

,order_id,account_id,bank_to,account_to,amount,k_symbol
0,29401,1,YZ,87144583,2452.0,SIPO
1,29402,2,ST,89597016,3372.7,UVER
2,29403,2,QR,13943797,7266.0,SIPO
3,29404,3,WX,83084338,1135.0,SIPO
4,29405,3,CD,24485939,327.0,NaN


In [4]:
df_account.isnull().sum()*100/df_account.shape[0]

account_id     0.0
district_id    0.0
frequency      0.0
date           0.0
dtype: float64

In [5]:
df_account.dtypes

account_id      int64
district_id     int64
frequency      object
date           object
dtype: object

- "POPLATEK MESICNE" stands for monthly issuance
- "POPLATEK TYDNE" stands for weekly issuance
- "POPLATEK PO OBRATU" stands for issuance after transaction

In [6]:
# Let's convert to English 
df_account['frequency'].replace('POPLATEK MESICNE', 'monthly', inplace=True)
df_account['frequency'].replace('POPLATEK TYDNE', 'weekly', inplace=True)
df_account['frequency'].replace('POPLATEK PO OBRATU', 'after_trans', inplace=True)
df_account.rename(columns = {'frequency':'stmt_frq'}, inplace=True) #statement frequency

In [7]:
# Convert date to a datetime type variable
df_account['date'] = pd.to_datetime(df_account['date'])

In [8]:
df_client.head()

,client_id,gender,birth_date,district_id
0,1,F,1970-12-13,18
1,2,M,1945-02-04,1
2,3,F,1940-10-09,1
3,4,M,1956-12-01,5
4,5,F,1960-07-03,5


In [9]:
df_client.isnull().sum()*100/df_client.shape[0]

client_id      0.0
gender         0.0
birth_date     0.0
district_id    0.0
dtype: float64

In [10]:
df_client.dtypes

client_id       int64
gender         object
birth_date     object
district_id     int64
dtype: object

In [11]:
# Convert date to a datetime type variable
df_client['birth_date'] = pd.to_datetime(df_client['birth_date'])

In [12]:
df_disp.head()

,disp_id,client_id,account_id,type
0,1,1,1,OWNER
1,2,2,2,OWNER
2,3,3,2,DISPONENT
3,4,4,3,OWNER
4,5,5,3,DISPONENT


In [13]:
df_disp.dtypes

disp_id        int64
client_id      int64
account_id     int64
type          object
dtype: object

In [14]:
df_order.head()

,order_id,account_id,bank_to,account_to,amount,k_symbol
0,29401,1,YZ,87144583,2452.0,SIPO
1,29402,2,ST,89597016,3372.7,UVER
2,29403,2,QR,13943797,7266.0,SIPO
3,29404,3,WX,83084338,1135.0,SIPO
4,29405,3,CD,24485939,327.0,NaN


In [15]:
df_order.isnull().sum()*100/df_order.shape[0]

order_id       0.000000
account_id     0.000000
bank_to        0.000000
account_to     0.000000
amount         0.000000
k_symbol      21.310462
dtype: float64

In [16]:
df_order.dtypes

order_id        int64
account_id      int64
bank_to        object
account_to      int64
amount        float64
k_symbol       object
dtype: object

- "POJISTNE" stands for insurance payment
- "SIPO" stands for household
- "LEASING" stands for leasing
- "UVER" stands for loan payment

In [17]:
# Let's convert k_symbol to English
df_order['k_symbol'].replace('POJISTNE', 'insurance', inplace=True)
df_order['k_symbol'].replace('SIPO', 'household', inplace=True)
df_order['k_symbol'].replace('LEASING', 'leasing', inplace=True)
df_order['k_symbol'].replace('UVER', 'loan', inplace=True)
df_order['k_symbol'].replace(np.nan, 'unknown', inplace=True)
df_order.rename(columns = {'k_symbol':'order_payment_type'}, inplace=True) 

In [18]:
df_trans.head()

,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account
0,1,1,1995-03-24,PRIJEM,VKLAD,1000,1000,NaN,NaN,NaN
1,5,1,1995-04-13,PRIJEM,PREVOD Z UCTU,3679,4679,NaN,AB,41403269.0
2,6,1,1995-05-13,PRIJEM,PREVOD Z UCTU,3679,20977,NaN,AB,41403269.0
3,7,1,1995-06-13,PRIJEM,PREVOD Z UCTU,3679,26835,NaN,AB,41403269.0
4,8,1,1995-07-13,PRIJEM,PREVOD Z UCTU,3679,30415,NaN,AB,41403269.0


In [19]:
df_trans.isnull().sum()*100/df_trans.shape[0]

#bank & account have over 70% of rows missing - won't be using

trans_id       0.000000
account_id     0.000000
date           0.000000
type           0.000000
operation     17.335088
amount         0.000000
balance        0.000000
k_symbol      45.618847
bank          74.107467
account       72.036031
dtype: float64

In [20]:
df_trans['operation'].unique()

array(['VKLAD', 'PREVOD Z UCTU', 'PREVOD NA UCET', 'VYBER',
       'VYBER KARTOU', nan], dtype=object)

In [21]:
df_trans['k_symbol'].unique()

array([nan, 'SIPO', 'SLUZBY', 'UVER', ' ', 'POJISTNE', 'DUCHOD', 'UROK',
       'SANKC. UROK'], dtype=object)

In [22]:
df_trans.dtypes

trans_id        int64
account_id      int64
date           object
type           object
operation      object
amount          int64
balance         int64
k_symbol       object
bank           object
account       float64
dtype: object

In [23]:
# Convert date to a datetime type variable
df_trans['date'] = pd.to_datetime(df_trans['date'])

`type` - type of transaction
- "PRIJEM" stands for credit
- "VYDAJ" stands for withdrawal

`operation` - mode of transaction
- "VYBER KARTOU" credit card withdrawal
- "VKLAD" credit in cash
- "PREVOD Z UCTU" collection from another bank
- "VYBER" withdrawal in cash
- "PREVOD NA UCET" remittance to another bank

`k_symbol` - characterization of transaction
- "POJISTNE" stands for insurrance payment
- "SLUZBY" stands for payment for statement
- "UROK" stands for interest credited
- "SANKC. UROK" sanction interest if negative balance
- "SIPO" stands for household
- "DUCHOD" stands for old-age pension
- "UVER" stands for loan payment

In [24]:
# Replace names for type
df_trans['type'].replace('PRIJEM', 'credit', inplace=True)
df_trans['type'].replace('VYDAJ', 'withdrawal', inplace=True)
df_trans.rename(columns = {'type':'trans_type'}, inplace=True) 

# Replace names for operation
df_trans['operation'].replace('VYBER KARTOU', 'cc_withdrawal', inplace=True) #credit card withdrawal
df_trans['operation'].replace('VKLAD', 'c_cash', inplace=True) #credit in cash
df_trans['operation'].replace('PREVOD Z UCTU', 'col_bank', inplace=True) #collection from another bank
df_trans['operation'].replace('VYBER', 'withdrawal_c', inplace=True) #withdrawal in cash
df_trans['operation'].replace('PREVOD NA UCET', 'remittance', inplace=True) #withdrawal in cash
df_trans['operation'].replace(np.nan, 'unknown', inplace=True)

# Replace names for k_symbol
df_trans['k_symbol'].replace('POJISTNE', 'insurance', inplace=True) 
df_trans['k_symbol'].replace('SLUZBY', 'statement', inplace=True) 
df_trans['k_symbol'].replace('UROK', 'int_cred', inplace=True) #interest credited
df_trans['k_symbol'].replace('SANKC. UROK', 'sanc_int', inplace=True) #sanction interest if negative balance
df_trans['k_symbol'].replace('SIPO', 'household', inplace=True) 
df_trans['k_symbol'].replace('DUCHOD', 'pension', inplace=True) 
df_trans['k_symbol'].replace('UVER', 'loan', inplace=True) 
df_trans['k_symbol'].replace(np.nan, 'unknown', inplace=True)
df_trans['k_symbol'].replace(' ', 'unknown', inplace=True)
df_trans.rename(columns = {'k_symbol':'trans_payment_type'}, inplace=True) 

In [25]:
df_loan.head()

,loan_id,account_id,date,amount,duration,payments,status
0,4959,2,1994-01-05,80952,24,3373.0,A
1,4961,19,1996-04-29,30276,12,2523.0,B
2,4962,25,1997-12-08,30276,12,2523.0,A
3,4967,37,1998-10-14,318480,60,5308.0,D
4,4968,38,1998-04-19,110736,48,2307.0,C


In [26]:
df_loan.isnull().sum()*100/df_loan.shape[0]

loan_id       0.0
account_id    0.0
date          0.0
amount        0.0
duration      0.0
payments      0.0
status        0.0
dtype: float64

In [27]:
df_loan.dtypes

loan_id         int64
account_id      int64
date           object
amount          int64
duration        int64
payments      float64
status         object
dtype: object

In [28]:
# Convert date to a datetime type variable
df_loan['date'] = pd.to_datetime(df_loan['date'])

In [29]:
# Rename loan columns
df_loan = df_loan.rename(columns={'amount': 'loan_amount', 'duration':'loan_duration', 'payments':'monthly_loan_payment', 'status':'loan_status'})

In [30]:
df_card.head()

,card_id,disp_id,type,issued
0,1,9,gold,1998-10-16
1,2,19,classic,1998-03-13
2,3,41,gold,1995-09-03
3,4,42,classic,1998-11-26
4,5,51,junior,1995-04-24


In [31]:
df_card.isnull().sum()*100/df_card.shape[0]

card_id    0.0
disp_id    0.0
type       0.0
issued     0.0
dtype: float64

In [32]:
df_card.dtypes

card_id     int64
disp_id     int64
type       object
issued     object
dtype: object

In [33]:
df_card['issued'] = pd.to_datetime(df_card['issued'])

In [34]:
df_district.head()

,district_id,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,1,Hl.m. Praha,Prague,1204953,0,0,0,1,1,100.0,12541,0.2,0.43,167,85677.0,99107
1,2,Benesov,central Bohemia,88884,80,26,6,2,5,46.7,8507,1.6,1.85,132,2159.0,2674
2,3,Beroun,central Bohemia,75232,55,26,4,1,5,41.7,8980,1.9,2.21,111,2824.0,2813
3,4,Kladno,central Bohemia,149893,63,29,6,2,6,67.4,9753,4.6,5.05,109,5244.0,5892
4,5,Kolin,central Bohemia,95616,65,30,4,1,6,51.4,9307,3.8,4.43,118,2616.0,3040


In [35]:
df_district.isnull().sum()*100/df_card.shape[0]

district_id    0.000000
A2             0.000000
A3             0.000000
A4             0.000000
A5             0.000000
A6             0.000000
A7             0.000000
A8             0.000000
A9             0.000000
A10            0.000000
A11            0.000000
A12            0.112108
A13            0.000000
A14            0.000000
A15            0.112108
A16            0.000000
dtype: float64

In [36]:
df_district.dtypes

district_id      int64
A2              object
A3              object
A4               int64
A5               int64
A6               int64
A7               int64
A8               int64
A9               int64
A10            float64
A11              int64
A12            float64
A13            float64
A14              int64
A15            float64
A16              int64
dtype: object

In [37]:
df_district = df_district.rename(columns={'A2':'district_name', 
                                          'A3':'region', 
                                          'A4':'population', 
                                          'A5':'nmu_lt499',
                                          'A6':'nmu_500to1999', 
                                          'A7':'nmu_2000to9999', 
                                          'A8':'nmu_gt10000',
                                          'A9':'n_cty', 
                                          'A10':'ratio_urban', 
                                          'A11':'avg_salary', 
                                          'A12':'unemp_95', 
                                          'A13': 'unemp_96',
                                          'A14':'nentrep_p1000', 
                                          'A15':'ncrimes_95', 
                                          'A16':'ncrimes_96'})

In [39]:
# Time to merge different dateframes

df_final = pd.merge(df_account, df_disp, on = 'account_id') #shape: 5369, 7

In [40]:
df_final.head()

,account_id,district_id,stmt_frq,date,disp_id,client_id,type
0,1,18,monthly,1995-03-24,1,1,OWNER
1,2,1,monthly,1993-02-26,2,2,OWNER
2,2,1,monthly,1993-02-26,3,3,DISPONENT
3,3,5,monthly,1997-07-07,4,4,OWNER
4,3,5,monthly,1997-07-07,5,5,DISPONENT


In [41]:
df_final = pd.merge(df_final, df_district, on = 'district_id') # shape: 5369, 22

In [42]:
df_final.head()

,account_id,district_id,stmt_frq,date,disp_id,client_id,type,district_name,region,population,nmu_lt499,nmu_500to1999,nmu_2000to9999,nmu_gt10000,n_cty,ratio_urban,avg_salary,unemp_95,unemp_96,nentrep_p1000,ncrimes_95,ncrimes_96
0,1,18,monthly,1995-03-24,1,1,OWNER,Pisek,south Bohemia,70699,60,13,2,1,4,65.3,8968,2.8,3.35,131,1740.0,1910
1,343,18,monthly,1994-04-26,420,420,OWNER,Pisek,south Bohemia,70699,60,13,2,1,4,65.3,8968,2.8,3.35,131,1740.0,1910
2,413,18,monthly,1996-07-09,499,499,OWNER,Pisek,south Bohemia,70699,60,13,2,1,4,65.3,8968,2.8,3.35,131,1740.0,1910
3,431,18,monthly,1996-11-05,519,519,OWNER,Pisek,south Bohemia,70699,60,13,2,1,4,65.3,8968,2.8,3.35,131,1740.0,1910
4,568,18,monthly,1995-03-28,682,682,OWNER,Pisek,south Bohemia,70699,60,13,2,1,4,65.3,8968,2.8,3.35,131,1740.0,1910


In [43]:
df_final = pd.merge(df_final, df_client, on = 'client_id') #shape: 5369, 25

#can drop one of the district_id

In [44]:
df_final.head()

,account_id,district_id_x,stmt_frq,date,disp_id,client_id,type,district_name,region,population,nmu_lt499,nmu_500to1999,nmu_2000to9999,nmu_gt10000,n_cty,ratio_urban,avg_salary,unemp_95,unemp_96,nentrep_p1000,ncrimes_95,ncrimes_96,gender,birth_date,district_id_y
0,1,18,monthly,1995-03-24,1,1,OWNER,Pisek,south Bohemia,70699,60,13,2,1,4,65.3,8968,2.8,3.35,131,1740.0,1910,F,1970-12-13,18
1,343,18,monthly,1994-04-26,420,420,OWNER,Pisek,south Bohemia,70699,60,13,2,1,4,65.3,8968,2.8,3.35,131,1740.0,1910,M,1978-03-13,18
2,413,18,monthly,1996-07-09,499,499,OWNER,Pisek,south Bohemia,70699,60,13,2,1,4,65.3,8968,2.8,3.35,131,1740.0,1910,F,1935-07-08,18
3,431,18,monthly,1996-11-05,519,519,OWNER,Pisek,south Bohemia,70699,60,13,2,1,4,65.3,8968,2.8,3.35,131,1740.0,1910,M,1980-04-13,18
4,568,18,monthly,1995-03-28,682,682,OWNER,Pisek,south Bohemia,70699,60,13,2,1,4,65.3,8968,2.8,3.35,131,1740.0,1910,M,1979-10-21,18


In [45]:
df_final = pd.merge(df_final, df_card, on='disp_id', how='outer', suffixes=('_disp','_card')) #shape: 5369, 28

In [46]:
df_final.head()

,account_id,district_id_x,stmt_frq,date,disp_id,client_id,type_disp,district_name,region,population,nmu_lt499,nmu_500to1999,nmu_2000to9999,nmu_gt10000,n_cty,ratio_urban,avg_salary,unemp_95,unemp_96,nentrep_p1000,ncrimes_95,ncrimes_96,gender,birth_date,district_id_y,card_id,type_card,issued
0,1,18,monthly,1995-03-24,1,1,OWNER,Pisek,south Bohemia,70699,60,13,2,1,4,65.3,8968,2.8,3.35,131,1740.0,1910,F,1970-12-13,18,NaN,NaN,NaT
1,343,18,monthly,1994-04-26,420,420,OWNER,Pisek,south Bohemia,70699,60,13,2,1,4,65.3,8968,2.8,3.35,131,1740.0,1910,M,1978-03-13,18,NaN,NaN,NaT
2,413,18,monthly,1996-07-09,499,499,OWNER,Pisek,south Bohemia,70699,60,13,2,1,4,65.3,8968,2.8,3.35,131,1740.0,1910,F,1935-07-08,18,NaN,NaN,NaT
3,431,18,monthly,1996-11-05,519,519,OWNER,Pisek,south Bohemia,70699,60,13,2,1,4,65.3,8968,2.8,3.35,131,1740.0,1910,M,1980-04-13,18,95.0,junior,1998-08-01
4,568,18,monthly,1995-03-28,682,682,OWNER,Pisek,south Bohemia,70699,60,13,2,1,4,65.3,8968,2.8,3.35,131,1740.0,1910,M,1979-10-21,18,NaN,NaN,NaT


In [47]:
df_final = pd.merge(df_final, df_loan, on='account_id', how='inner', suffixes=('_account','_loan')) #shape: 827, 34

#We only keep the accounts with loans 

In [48]:
len(df_final['client_id'].unique()) 

# There are 827 unique clients with loans

827

In [49]:
len(df_final['account_id'].unique()) 

# This makes sense as the semi anonymized dataset contains 606 successful and 76 not successful loans.

682

In [50]:
df_final.head()

,account_id,district_id_x,stmt_frq,date_account,disp_id,client_id,type_disp,district_name,region,population,nmu_lt499,nmu_500to1999,nmu_2000to9999,nmu_gt10000,n_cty,ratio_urban,avg_salary,unemp_95,unemp_96,nentrep_p1000,ncrimes_95,ncrimes_96,gender,birth_date,district_id_y,card_id,type_card,issued,loan_id,date_loan,loan_amount,loan_duration,monthly_loan_payment,loan_status
0,2350,18,monthly,1996-08-04,2841,2841,OWNER,Pisek,south Bohemia,70699,60,13,2,1,4,65.3,8968,2.8,3.35,131,1740.0,1910,F,1973-04-10,18,446.0,classic,1998-06-04,5451,1997-05-11,159744,48,3328.0,C
1,9156,18,monthly,1997-09-14,10963,11271,OWNER,Pisek,south Bohemia,70699,60,13,2,1,4,65.3,8968,2.8,3.35,131,1740.0,1910,M,1937-12-08,18,NaN,NaN,NaT,6856,1998-12-01,163332,36,4537.0,C
2,10973,18,weekly,1993-04-20,13182,13490,OWNER,Pisek,south Bohemia,70699,60,13,2,1,4,65.3,8968,2.8,3.35,131,1740.0,1910,F,1969-05-25,18,NaN,NaN,NaT,7235,1993-10-13,154416,48,3217.0,A
3,2,1,monthly,1993-02-26,2,2,OWNER,Hl.m. Praha,Prague,1204953,0,0,0,1,1,100.0,12541,0.2,0.43,167,85677.0,99107,M,1945-02-04,1,NaN,NaN,NaT,4959,1994-01-05,80952,24,3373.0,A
4,2,1,monthly,1993-02-26,3,3,DISPONENT,Hl.m. Praha,Prague,1204953,0,0,0,1,1,100.0,12541,0.2,0.43,167,85677.0,99107,F,1940-10-09,1,NaN,NaN,NaT,4959,1994-01-05,80952,24,3373.0,A


In [51]:
df_trans.head()

,trans_id,account_id,date,trans_type,operation,amount,balance,trans_payment_type,bank,account
0,1,1,1995-03-24,credit,c_cash,1000,1000,unknown,NaN,NaN
1,5,1,1995-04-13,credit,col_bank,3679,4679,unknown,AB,41403269.0
2,6,1,1995-05-13,credit,col_bank,3679,20977,unknown,AB,41403269.0
3,7,1,1995-06-13,credit,col_bank,3679,26835,unknown,AB,41403269.0
4,8,1,1995-07-13,credit,col_bank,3679,30415,unknown,AB,41403269.0


In [52]:
df_trans_account = pd.merge(df_trans, df_final[['account_id','date_loan']], on = 'account_id') 

#shape: 233627, 11 transactions for accounts with loans

In [53]:
df_trans_account.head()

,trans_id,account_id,date,trans_type,operation,amount,balance,trans_payment_type,bank,account,date_loan
0,276,2,1993-02-26,credit,c_cash,1100,1100,unknown,NaN,NaN,1994-01-05
1,276,2,1993-02-26,credit,c_cash,1100,1100,unknown,NaN,NaN,1994-01-05
2,279,2,1993-03-12,credit,col_bank,20236,21336,unknown,ST,66487163.0,1994-01-05
3,279,2,1993-03-12,credit,col_bank,20236,21336,unknown,ST,66487163.0,1994-01-05
4,280,2,1993-04-12,credit,col_bank,20236,45286,unknown,ST,66487163.0,1994-01-05


In [54]:
#Drop duplicate transactions
df_trans_account = df_trans_account.drop_duplicates()

In [56]:
df_trans_account.head()

,trans_id,account_id,date,trans_type,operation,amount,balance,trans_payment_type,bank,account,date_loan
0,276,2,1993-02-26,credit,c_cash,1100,1100,unknown,NaN,NaN,1994-01-05
2,279,2,1993-03-12,credit,col_bank,20236,21336,unknown,ST,66487163.0,1994-01-05
4,280,2,1993-04-12,credit,col_bank,20236,45286,unknown,ST,66487163.0,1994-01-05
6,281,2,1993-05-12,credit,col_bank,20236,54631,unknown,ST,66487163.0,1994-01-05
8,282,2,1993-06-12,credit,col_bank,30354,67530,unknown,ST,66487163.0,1994-01-05


In [57]:
# Get the difference between the date of the loan and the date of the transaction
df_trans_account['date_diff'] = (df_trans_account['date_loan'] - df_trans_account['date']) 

In [58]:
df_trans_account.head()

,trans_id,account_id,date,trans_type,operation,amount,balance,trans_payment_type,bank,account,date_loan,date_diff
0,276,2,1993-02-26,credit,c_cash,1100,1100,unknown,NaN,NaN,1994-01-05,313 days
2,279,2,1993-03-12,credit,col_bank,20236,21336,unknown,ST,66487163.0,1994-01-05,299 days
4,280,2,1993-04-12,credit,col_bank,20236,45286,unknown,ST,66487163.0,1994-01-05,268 days
6,281,2,1993-05-12,credit,col_bank,20236,54631,unknown,ST,66487163.0,1994-01-05,238 days
8,282,2,1993-06-12,credit,col_bank,30354,67530,unknown,ST,66487163.0,1994-01-05,207 days


In [59]:
df_trans_account.shape

(191556, 12)

In [60]:
# Let's drop transactions that occured after the loan date
df_trans_account.drop(df_trans_account[df_trans_account['date_diff'] < datetime.timedelta(0)].index, inplace=True) 

In [61]:
df_trans_account.shape

(54860, 12)

In [63]:
df_trans_type_counts = df_trans_account.groupby('account_id')['trans_type'].value_counts().to_frame()
df_operation_counts = df_trans_account.groupby('account_id')['operation'].value_counts().to_frame()
df_payment_type_counts = df_trans_account.groupby('account_id')['trans_payment_type'].value_counts().to_frame()

In [64]:
df_trans_type_counts.index = df_trans_type_counts.index.set_names(['account_id', 'transaction_type'])
df_operation_counts.index = df_operation_counts.index.set_names(['account_id', 'operation_type'])
df_payment_type_counts.index = df_payment_type_counts.index.set_names(['account_id', 'payment_type'])

In [65]:
df_trans_type_counts.reset_index(inplace=True)
df_operation_counts.reset_index(inplace=True)
df_payment_type_counts.reset_index(inplace=True)

In [66]:
df_trans_type_counts = df_trans_type_counts.pivot(index='account_id', columns='transaction_type', values='trans_type').fillna(0).reset_index(inplace=False)
df_operation_counts = df_operation_counts.pivot(index='account_id', columns='operation_type', values='operation').fillna(0).reset_index(inplace=False)
df_payment_type_counts = df_payment_type_counts.pivot(index='account_id', columns='payment_type', values='trans_payment_type').fillna(0).reset_index(inplace=False)

In [67]:
df_trans_type_counts.columns = ['num_trans_' + str(col) for col in df_trans_type_counts.columns]
df_operation_counts.columns = ['num_ops_' + str(col) for col in df_operation_counts.columns]
df_payment_type_counts.columns = ['num_pay_' + str(col) for col in df_payment_type_counts.columns]

In [68]:
df_trans_type_counts.rename(columns={'num_trans_account_id':'account_id'}, inplace=True)
df_operation_counts.rename(columns={'num_ops_account_id':'account_id'}, inplace=True)
df_payment_type_counts.rename(columns={'num_pay_account_id':'account_id'}, inplace=True)

In [72]:
df_payment_type_counts['account_id'].nunique()

682

In [275]:
#Create new transaction counts data frame 
counts_dataframes = [df_trans_type_counts, df_operation_counts, df_payment_type_counts]
df_counts = reduce(lambda left,right: pd.merge(left,right,on='account_id'), counts_dataframes)#shape: 682,17

In [276]:
df_final = pd.merge(df_final, df_counts, on = 'account_id') #shape: 827, 50

In [277]:
df_final.shape

(827, 50)

In [278]:
df_final.head(1)

,account_id,district_id_x,stmt_frq,date_account,disp_id,client_id,type_disp,district_name,region,population,nmu_lt499,nmu_500to1999,nmu_2000to9999,nmu_gt10000,n_cty,ratio_urban,avg_salary,unemp_95,unemp_96,nentrep_p1000,ncrimes_95,ncrimes_96,gender,birth_date,district_id_y,card_id,type_card,issued,loan_id,date_loan,loan_amount,loan_duration,monthly_loan_payment,loan_status,num_trans_VYBER,num_trans_credit,num_trans_withdrawal,num_ops_c_cash,num_ops_cc_withdrawal,num_ops_col_bank,num_ops_remittance,num_ops_unknown,num_ops_withdrawal_c,num_pay_household,num_pay_insurance,num_pay_int_cred,num_pay_loan,num_pay_sanc_int,num_pay_statement,num_pay_unknown
0,2350,18,monthly,1996-08-04,2841,2841,OWNER,Pisek,south Bohemia,70699,60,13,2,1,4,65.3,8968,2.8,3.35,131,1740.0,1910,F,1973-04-10,18,446.0,classic,1998-06-04,5451,1997-05-11,159744,48,3328.0,C,0.0,15.0,13.0,10.0,0.0,0.0,3.0,5.0,10.0,2.0,0.0,5.0,0.0,0.0,2.0,19.0


Most lenders ask to see at least two-three months' worth of statements before they issue you a loan.

Let's take a look at the account balance before the loan date for the past 3 months.

In [279]:
df_trans_account_copy = df_trans_account.copy()

In [280]:
df_trans_account_copy.shape

(54860, 12)

In [281]:
# Let's take a look at transactions within different time frames upto 3 months
df_trans_account_30 = df_trans_account_copy.copy()
df_trans_account_60 = df_trans_account_copy.copy()
df_trans_account_90 = df_trans_account_copy.copy()

In [282]:
# Reducing transactions by months (30, 60, 90 days)
df_trans_account_30.drop(df_trans_account_30[df_trans_account_30['date_diff'] > datetime.timedelta(30)].index, inplace=True)
df_trans_account_60.drop(df_trans_account_60[df_trans_account_60['date_diff'] > datetime.timedelta(60)].index, inplace=True)
df_trans_account_90.drop(df_trans_account_90[df_trans_account_90['date_diff'] > datetime.timedelta(90)].index, inplace=True)

In [283]:
df_trans_account_30.shape

(4960, 12)

In [284]:
df_trans_account_90.shape

#Makes sense as there are more transaction with more time

(14316, 12)

In [285]:
mon_1_balance = df_trans_account_30.groupby('account_id')['balance'].agg(['min','max','mean','count']).reset_index()
mon_2_balance = df_trans_account_60.groupby('account_id')['balance'].agg(['min','max','mean','count']).reset_index()
mon_3_balance = df_trans_account_90.groupby('account_id')['balance'].agg(['min','max','mean','count']).reset_index()

In [286]:
mon_1_balance.rename(columns = {'min':'min1','max':'max1','mean':'mean1','count':'count1'}, inplace=True)
mon_2_balance.rename(columns = {'min':'min2','max':'max2','mean':'mean2','count':'count2'}, inplace=True)
mon_3_balance.rename(columns = {'min':'min3','max':'max3','mean':'mean3','count':'count3'}, inplace=True)

In [287]:
#Create new client data frame with different balance statistics for different time frames
balance_dataframes = [df_final, mon_1_balance, mon_2_balance, mon_3_balance]
df_final = reduce(lambda left,right: pd.merge(left,right,on='account_id'), balance_dataframes)#shape:827,62 

In [288]:
# Let's focus on the owners of the account but we'll keep a count if there are multiple people on an account 

df_num_clients = df_final.groupby('account_id', as_index=False)['type_disp'].count().rename(columns={'type_disp':'num_clients'})

In [289]:
df_num_clients.shape

(682, 2)

In [290]:
df_final = pd.merge(df_final, df_num_clients, on = 'account_id')

In [291]:
df_final.head()

,account_id,district_id_x,stmt_frq,date_account,disp_id,client_id,type_disp,district_name,region,population,nmu_lt499,nmu_500to1999,nmu_2000to9999,nmu_gt10000,n_cty,ratio_urban,avg_salary,unemp_95,unemp_96,nentrep_p1000,ncrimes_95,ncrimes_96,gender,birth_date,district_id_y,card_id,type_card,issued,loan_id,date_loan,loan_amount,loan_duration,monthly_loan_payment,loan_status,num_trans_VYBER,num_trans_credit,num_trans_withdrawal,num_ops_c_cash,num_ops_cc_withdrawal,num_ops_col_bank,num_ops_remittance,num_ops_unknown,num_ops_withdrawal_c,num_pay_household,num_pay_insurance,num_pay_int_cred,num_pay_loan,num_pay_sanc_int,num_pay_statement,num_pay_unknown,min1,max1,mean1,count1,min2,max2,mean2,count2,min3,max3,mean3,count3,num_clients
0,2350,18,monthly,1996-08-04,2841,2841,OWNER,Pisek,south Bohemia,70699,60,13,2,1,4,65.3,8968,2.8,3.35,131,1740.0,1910,F,1973-04-10,18,446.0,classic,1998-06-04,5451,1997-05-11,159744,48,3328.0,C,0.0,15.0,13.0,10.0,0.0,0.0,3.0,5.0,10.0,2.0,0.0,5.0,0.0,0.0,2.0,19.0,53598,79188,69002.000000,5,41964,105954,70175.285714,14,39875,105954,68155.266667,15,1
1,9156,18,monthly,1997-09-14,10963,11271,OWNER,Pisek,south Bohemia,70699,60,13,2,1,4,65.3,8968,2.8,3.35,131,1740.0,1910,M,1937-12-08,18,NaN,NaN,NaT,6856,1998-12-01,163332,36,4537.0,C,4.0,34.0,28.0,11.0,0.0,0.0,0.0,23.0,32.0,0.0,0.0,23.0,0.0,0.0,6.0,37.0,17501,23801,18828.200000,5,2777,28615,16541.461538,13,2777,67680,24103.263158,19,1
2,10973,18,weekly,1993-04-20,13182,13490,OWNER,Pisek,south Bohemia,70699,60,13,2,1,4,65.3,8968,2.8,3.35,131,1740.0,1910,F,1969-05-25,18,NaN,NaN,NaT,7235,1993-10-13,154416,48,3217.0,A,2.0,16.0,14.0,10.0,0.0,0.0,0.0,6.0,16.0,0.0,0.0,6.0,0.0,0.0,4.0,22.0,20150,56033,42202.375000,8,20150,56033,41657.083333,12,20150,70024,44007.176471,17,1
3,2,1,monthly,1993-02-26,2,2,OWNER,Hl.m. Praha,Prague,1204953,0,0,0,1,1,100.0,12541,0.2,0.43,167,85677.0,99107,M,1945-02-04,1,NaN,NaN,NaT,4959,1994-01-05,80952,24,3373.0,A,3.0,22.0,30.0,2.0,0.0,10.0,6.0,10.0,27.0,6.0,0.0,10.0,0.0,0.0,6.0,33.0,16859,47213,29349.888889,9,16859,47213,30791.533333,15,16859,47213,29830.000000,20,2
4,2,1,monthly,1993-02-26,3,3,DISPONENT,Hl.m. Praha,Prague,1204953,0,0,0,1,1,100.0,12541,0.2,0.43,167,85677.0,99107,F,1940-10-09,1,NaN,NaN,NaT,4959,1994-01-05,80952,24,3373.0,A,3.0,22.0,30.0,2.0,0.0,10.0,6.0,10.0,27.0,6.0,0.0,10.0,0.0,0.0,6.0,33.0,16859,47213,29349.888889,9,16859,47213,30791.533333,15,16859,47213,29830.000000,20,2


In [292]:
df_final_owner = df_final[df_final['type_disp']=='OWNER']

In [293]:
df_final_owner.shape

(682, 63)

In [294]:
df_final_owner.to_csv('df_final_owner.csv')